In [249]:
import pandas as pd
import numpy as np
import re

In [32]:
demand_df=pd.read_csv("Data/demand_set.csv")
demand_df

,student,demand
0,0,0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0
1,0,0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0
2,0,0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0
3,0,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
4,0,0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
...,...,...
295,49,0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
296,49,0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
297,49,0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
298,49,0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0


In [78]:
pd.set_option('display.max_rows', None)

In [275]:
allocation_df = pd.DataFrame(index=range(301), columns=['student','allocation'])
class_dict={}
budget_dict={}
try:
    with open("Data/RESULT2023-12-21 00_41_38.618632.txt", "r") as file:
        lines = file.readlines()[:50]
        for line_number, line in enumerate(lines, start=1):
            print(f"Line {line_number}: {line.strip()}")
            student_num=(str(line[9])+str(line[10])).strip()
            extracted_numbers = re.findall(r'\d+', line[11:30])
            print('extract:', extracted_numbers)
            class_dict[line_number]=extracted_numbers
            match = re.search(r'budget:\s*([\d.]+)', line)
            if match:
                # Extracted numbers after "budget"
                budget_number = float(match.group(1))
                print("Extracted budget number :",line_number, budget_number)
                budget_dict[line_number-1]=budget_number
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")
    

Line 1: student: 0 classes: [[10]] budget: 1.00005
extract: ['10']
Extracted budget number : 1 1.00005
Line 2: student: 1 classes: [[0, 4]] budget: 0.99
extract: ['0', '4']
Extracted budget number : 2 0.99
Line 3: student: 2 classes: [[10]] budget: 1.00005
extract: ['10']
Extracted budget number : 3 1.00005
Line 4: student: 3 classes: [[9]] budget: 0.9901
extract: ['9']
Extracted budget number : 4 0.9901
Line 5: student: 4 classes: [[10]] budget: 1.00005
extract: ['10']
Extracted budget number : 5 1.00005
Line 6: student: 5 classes: [[5]] budget: 0.99
extract: ['5']
Extracted budget number : 6 0.99
Line 7: student: 6 classes: [[4, 6]] budget: 0.99
extract: ['4', '6']
Extracted budget number : 7 0.99
Line 8: student: 7 classes: [[8]] budget: 0.99
extract: ['8']
Extracted budget number : 8 0.99
Line 9: student: 8 classes: [[2, 4]] budget: 0.99
extract: ['2', '4']
Extracted budget number : 9 0.99
Line 10: student: 9 classes: [[6]] budget: 0.99
extract: ['6']
Extracted budget number : 10 0

In [276]:
allocations_list=[]
for key, val in class_dict.items():
    allocations_list.append(val)

for i in range(len(allocations_list)):
    for j in range(len(allocations_list[i])):
        allocations_list[i][j] = int(allocations_list[i][j])
            
ohe_allocations_list = [[1 if i in sublist else 0 for i in range(max(max(sublist, default=0) for sublist in allocations_list) + 1)] for sublist in allocations_list]

        
ohe_allocations_list
ohe_allocations_dict={}
for i,alloc in enumerate(ohe_allocations_list):
    ohe_allocations_dict[i]=alloc
    
student_demand_bundles_dict = {}
for index, row in demand_df.iterrows():
    student_id = row['student']
    demand_bundle = row['demand']
    string_values = demand_bundle.split()
    int_values = [int(float(value)) for value in string_values]

    if student_id not in student_demand_bundles_dict:
        student_demand_bundles_dict[student_id] = []

    student_demand_bundles_dict[student_id].append(int_values)

ohe_demand_1_dict={}
ohe_demand_2_dict={}
ohe_demand_3_dict={}
ohe_demand_4_dict={}
ohe_demand_5_dict={}
ohe_demand_6_dict={}
i=0
for key, val in student_demand_bundles_dict.items():
    ohe_demand_1_dict[key]=val[0]
    ohe_demand_2_dict[key]=val[1]
    ohe_demand_3_dict[key]=val[2]
    ohe_demand_4_dict[key]=val[3]
    ohe_demand_5_dict[key]=val[4]
    ohe_demand_6_dict[key]=val[5]


In [277]:
overlap_df=pd.DataFrame(index=range(50), columns=['demand_1','demand_2','demand_3','demand_4','demand_5','demand_6'])


In [290]:
if ohe_demand_1_dict == ohe_allocations_dict:
    for key, val in ohe_demand_1_dict.items():
        overlap_df.at[key,"demand_1"]=val
else:
    for key, val in ohe_demand_1_dict.items():
        overlap_df.at[key,"demand_1"]=-100
        
if ohe_demand_2_dict == ohe_allocations_dict:
    for key, val in ohe_demand_2_dict.items():
        overlap_df.at[key,"demand_2"]=val
else:
    for key, val in ohe_demand_2_dict.items():
        overlap_df.at[key,"demand_2"]=-100
        
if ohe_demand_3_dict == ohe_allocations_dict:
    for key, val in ohe_demand_3_dict.items():
        overlap_df.at[key,"demand_3"]=val
else:
    for key, val in ohe_demand_3_dict.items():
        overlap_df.at[key,"demand_3"]=-100
        
if ohe_demand_4_dict == ohe_allocations_dict:
    for key, val in ohe_demand_4_dict.items():
        overlap_df.at[key,"demand_4"]=val
else:
    for key, val in ohe_demand_4_dict.items():
        overlap_df.at[key,"demand_4"]=-100
        
if ohe_demand_5_dict == ohe_allocations_dict:
    for key, val in ohe_demand_5_dict.items():
        overlap_df.at[key,"demand_5"]=val
else:
    for key, val in ohe_demand_5_dict.items():
        overlap_df.at[key,"demand_5"]=-100
        
if ohe_demand_6_dict == ohe_allocations_dict:
    for key, val in ohe_demand_6_dict.items():
        overlap_df.at[key,"demand_6"]=val
else:
    for key, val in ohe_demand_6_dict.items():
        overlap_df.at[key,"demand_6"]=-100


In [297]:
envy_df=pd.DataFrame(index=range(50), columns=['demand_1','demand_2','demand_3','demand_4','demand_5','demand_6','budget','allocation','prefered_bundle'])


In [303]:
def points_tracker(transformed_allocations_dict,demand_dict,column_name_entry):
    overlap_dict_1 = {key: [(i, value) for i, value in enumerate(demand_dict[key]) if value in transformed_allocations_dict[key]] for key in demand_dict.keys() & transformed_allocations_dict.keys()}
    points_dict={}
    for key, val in overlap_dict_1.items():
        for i,item in enumerate(val):
            if item[0]==0:
                if not bool(points_dict.get(key, {})):
                    points_dict[key]=3
                else:
                    points_dict[key]+=3
            elif item[0]==1:
                if not bool(points_dict.get(key, {})):
                    points_dict[key]=2
                else:
                    points_dict[key]+=2
            elif item[0]==2:
                if not bool(points_dict.get(key, {})):
                    points_dict[key]=1
                else:
                    points_dict[key]+=1
            else:
                if not bool(points_dict.get(key, {})):
                    points_dict[key]=0
                else:
                    points_dict[key]+=0
    
    for key, val in  points_dict.items():
        envy_df.at[key,column_name_entry]=val
    
    return envy_df

In [304]:
demand_1_dict={}
demand_2_dict={}
demand_3_dict={}
demand_4_dict={}
demand_5_dict={}
demand_6_dict={}

In [305]:
transformed_allocations_dict={}
for key, value_list in ohe_allocations_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    transformed_allocations_dict[key] = indices_of_ones
    
for key, value_list in ohe_demand_1_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    demand_1_dict[key] = indices_of_ones
    
for key, value_list in ohe_demand_2_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    demand_2_dict[key] = indices_of_ones

for key, value_list in ohe_demand_3_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    demand_3_dict[key] = indices_of_ones
    

for key, value_list in ohe_demand_4_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    demand_4_dict[key] = indices_of_ones

for key, value_list in ohe_demand_5_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    demand_5_dict[key] = indices_of_ones

for key, value_list in ohe_demand_6_dict.items():
    indices_of_ones = [index for index, value in enumerate(value_list) if value == 1]
    demand_6_dict[key] = indices_of_ones



In [306]:
df=points_tracker(transformed_allocations_dict,demand_1_dict,'demand_1')
df=points_tracker(transformed_allocations_dict,demand_2_dict,'demand_2')
df=points_tracker(transformed_allocations_dict,demand_3_dict,'demand_3')
df=points_tracker(transformed_allocations_dict,demand_4_dict,'demand_4')
df=points_tracker(transformed_allocations_dict,demand_5_dict,'demand_5')
df=points_tracker(transformed_allocations_dict,demand_6_dict,'demand_6')
df

,demand_1,demand_2,demand_3,demand_4,demand_5,demand_6,budget,allocation,prefered_bundle
0,1,2,2,3,NaN,NaN,1.00005,[10],"[4, 6, 10]"
1,5,3,NaN,5,3,3,0.99,"[0, 4]","[0, 4, 10]"
2,1,2,2,3,NaN,NaN,1.00005,[10],"[4, 6, 10]"
3,1,2,3,NaN,NaN,NaN,0.9901,[9],"[1, 6, 9]"
4,1,2,2,3,NaN,NaN,1.00005,[10],"[0, 2, 10]"
5,1,2,2,3,NaN,NaN,0.99,[5],"[0, 4, 5]"
6,5,3,5,NaN,3,3,0.99,"[4, 6]","[4, 6, 9]"
7,1,2,NaN,3,NaN,NaN,0.99,[8],"[4, 5, 8]"
8,5,3,5,NaN,3,3,0.99,"[2, 4]","[2, 4, 6]"
9,1,2,3,NaN,NaN,NaN,0.99,[6],"[0, 5, 6]"


In [311]:
for key, val in budget_dict.items():
    df.at[key,'budget']=val
    df.at[key,'allocation']=transformed_allocations_dict[key]
    df.at[key,'prefered_bundle']=demand_1_dict[key]
df

,demand_1,demand_2,demand_3,demand_4,demand_5,demand_6,budget,allocation,prefered_bundle
0,1,2,2,3,NaN,NaN,1.00005,[10],"[4, 6, 10]"
1,5,3,NaN,5,3,3,0.99,"[0, 4]","[0, 4, 10]"
2,1,2,2,3,NaN,NaN,1.00005,[10],"[4, 6, 10]"
3,1,2,3,NaN,NaN,NaN,0.9901,[9],"[1, 6, 9]"
4,1,2,2,3,NaN,NaN,1.00005,[10],"[0, 2, 10]"
5,1,2,2,3,NaN,NaN,0.99,[5],"[0, 4, 5]"
6,5,3,5,NaN,3,3,0.99,"[4, 6]","[4, 6, 9]"
7,1,2,NaN,3,NaN,NaN,0.99,[8],"[4, 5, 8]"
8,5,3,5,NaN,3,3,0.99,"[2, 4]","[2, 4, 6]"
9,1,2,3,NaN,NaN,NaN,0.99,[6],"[0, 5, 6]"


[[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]